In [1]:
import pandas as pd
import numpy as np
from lr.text_processing.util import simple_pre_process_text_df
from multiprocessing import Pool
from functools import reduce
from collections import Counter

In [2]:
syn_noum = pd.read_csv("data/mnli/syn_noun.csv")
keys = syn_noum.key.to_list()
values = syn_noum.value.to_list()
syn_noum_dict = {k:v for k,v in zip(syn_noum.key, syn_noum.value)}
syn_noum_dict_inv = {v:k for k,v in zip(syn_noum.key, syn_noum.value)}

def check_key(s):
    return [w for w in s.split(" ") if w in keys] 

def check_key_df(df):    
    df.loc[:, "w_trans"] = df["text"].map(check_key)
    df.loc[:, "n_trans"] = df["w_trans"].map(lambda x : len(x))
    return df
    
def parallelize(df, func, n_cores):
    """
    general fucntion to parallelize a function applied to
    a df
    """
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [3]:
df = pd.read_csv("data/mnli/test.csv").dropna()
simple_pre_process_text_df(df, "premise")
simple_pre_process_text_df(df, "hypothesis")
df.loc[:, "text"] = df.premise + " " + df.hypothesis
df.loc[:, "size"] = df["text"].map(lambda x: len(x.split(" ")))
df = parallelize(df, func=check_key_df, n_cores=20)

df_neu  = df.query("label=='neutral'")
df_con  = df.query("label=='contradiction'")
df_ent  = df.query("label=='entailment'")

df_p = pd.read_csv("data/mnli/test_p_h_syn_noun.csv").dropna()
df_p.loc[:, "text"] = df_p.premise + " " + df_p.hypothesis
df_p.loc[:, "size"] = df_p["text"].map(lambda x: len(x.split(" ")))
df_p = parallelize(df_p, func=check_key_df, n_cores=20)

df_p_neu  = df_p.query("label=='neutral'")
df_p_con  = df_p.query("label=='contradiction'")
df_p_ent  = df_p.query("label=='entailment'")

In [4]:
df.shape[0], df_p.shape[0]

(20000, 19647)

In [5]:
(df["n_trans"]>0).mean()*100

91.925

In [6]:
df["n_trans"].mean()

4.3529

## Sizes (Original)

In [7]:
avg_total = df[["size"]].describe().loc["mean"].values[0]
avg_con = df_con[["size"]].describe().loc["mean"].values[0]
avg_neu = df_neu[["size"]].describe().loc["mean"].values[0]
avg_ent = df_ent[["size"]].describe().loc["mean"].values[0]
results = [avg_total, avg_con, avg_neu, avg_ent]
results = [np.round(i,2) for i in results]
columns = ["total", "contradiction", "neutral", "entailment"]
original = pd.DataFrame([results], columns=columns)
original

,total,contradiction,neutral,entailment
0,29.99,29.28,30.42,30.13


## Sizes (Transformed)

In [8]:
avg_total = df_p[["size"]].describe().loc["mean"].values[0]
avg_con = df_p_con[["size"]].describe().loc["mean"].values[0]
avg_neu = df_p_neu[["size"]].describe().loc["mean"].values[0]
avg_ent = df_p_ent[["size"]].describe().loc["mean"].values[0]
results = [avg_total, avg_con, avg_neu, avg_ent]
results = [np.round(i,2) for i in results]
columns = ["total", "contradiction", "neutral", "entailment"]
transformed = pd.DataFrame([results], columns=columns)
transformed

,total,contradiction,neutral,entailment
0,32.34,31.65,32.83,32.55


In [9]:
pd.concat([original, transformed]).pct_change()* 100

,total,contradiction,neutral,entailment
0,NaN,NaN,NaN,NaN
0,7.835945,8.094262,7.922419,8.031862


In [10]:
df.head(3)

,premise,hypothesis,label,text,size,w_trans,n_trans
0,the new rights are nice enough,everyone really likes the newest benefits,neutral,the new rights are nice enough everyone really...,13,[enough],1
1,this site includes a list of all award winners...,the government executive articles housed on th...,contradiction,this site includes a list of all award winners...,31,"[site, government, executive, government, exec...",6
2,uh i dont know i i have mixed emotions about h...,i like him for the most part but would still e...,entailment,uh i dont know i i have mixed emotions about h...,43,"[beat, part, enjoy, beat]",4


## Count transformations

In [11]:
avg_total = df["n_trans"].mean()
avg_con = df_con["n_trans"].mean()
avg_neu = df_neu["n_trans"].mean()
avg_ent = df_ent["n_trans"].mean()
results = [avg_total, avg_con, avg_neu, avg_ent]
results = [np.round(i,2) for i in results]
columns = ["total", "contradiction", "neutral", "entailment"]
n_tranforms = pd.DataFrame([results], columns=columns)
n_tranforms

,total,contradiction,neutral,entailment
0,4.35,4.22,4.39,4.41


## Check transformations

In [12]:
def lists2list(arr):
    return reduce(lambda x,y: x+y, arr)

def get_counter(arrays, n_cores):
    splits = np.array_split(arrays, n_cores)
    pool = Pool(n_cores)
    final_list = lists2list(pool.map(lists2list, splits))
    pool.close()
    pool.join()
    key_count = Counter(final_list)
    return key_count

count_total = get_counter(df["w_trans"].values, n_cores=20)
count_total.most_common(10)

[('been', 1226),
 ('said', 712),
 ('also', 679),
 ('children', 673),
 ('help', 670),
 ('information', 627),
 ('year', 623),
 ('need', 534),
 ('made', 507),
 ('before', 442)]

In [13]:
count_con = get_counter(df_con["w_trans"].values, n_cores=20)
count_con.most_common(10)

[('been', 393),
 ('said', 237),
 ('help', 206),
 ('children', 197),
 ('information', 193),
 ('year', 193),
 ('also', 183),
 ('need', 182),
 ('before', 155),
 ('made', 142)]

In [14]:
count_neu = get_counter(df_neu["w_trans"].values, n_cores=20)
count_neu.most_common(10)

[('been', 407),
 ('also', 239),
 ('children', 227),
 ('help', 226),
 ('said', 211),
 ('year', 200),
 ('information', 196),
 ('need', 163),
 ('made', 157),
 ('world', 142)]

In [15]:
count_ent = get_counter(df_ent["w_trans"].values, n_cores=20)
count_ent.most_common(10)

[('been', 406),
 ('said', 254),
 ('also', 241),
 ('children', 235),
 ('help', 234),
 ('information', 227),
 ('year', 218),
 ('made', 197),
 ('need', 178),
 ('center', 152)]

## Artifacts analysis

In [16]:
count_gen = lambda w: lambda s: int(w in s.split(" "))
con_art = ["never", "no", "nothing", "any", "none"]
neu_art = ["also", "because", "popular", "many", "most"]
ent_art = ["some", "yes", "something", "sometimes", "various"]
con_art_t = [(w, syn_noum_dict[w]) for w in con_art if w in keys]
neu_art_t = [(w, syn_noum_dict[w]) for w in neu_art if w in keys]
ent_art_t = [(w, syn_noum_dict[w]) for w in ent_art if w in keys]

### Countradition

In [17]:
con_art_t, [(syn_noum_dict_inv[w], w) for w in con_art if w in values]

([], [])

In [18]:
df_con.head(1)

,premise,hypothesis,label,text,size,w_trans,n_trans
1,this site includes a list of all award winners...,the government executive articles housed on th...,contradiction,this site includes a list of all award winners...,31,"[site, government, executive, government, exec...",6


In [19]:
df_p_con.head(1)

,premise,hypothesis,label,text,size,w_trans,n_trans
1,this land site includes a list of all award wi...,the authorities executive director articles h...,contradiction,this land site includes a list of all award wi...,36,"[site, authorities, executive, director, autho...",8


In [20]:
for w in con_art:
    f_count = count_gen(w)
    df_con.loc[:, "{}_count".format(w)] = df_con["text"].map(f_count)
    df_p_con.loc[:, "{}_count".format(w)] = df_p_con["text"].map(f_count)
    print(w, np.round(df_con["{}_count".format(w)].mean()*100,2), np.round(df_p_con["{}_count".format(w)].mean()*100,2))

never 6.43 6.43
no 11.39 11.39
nothing 2.36 2.36
any 6.74 6.74
none 0.56 0.56


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


### Neutral

In [21]:
neu_art_t, [(syn_noum_dict_inv[w], w) for w in neu_art if w in values]

([('also', 'too')], [])

In [22]:
for w in neu_art:
    f_count = count_gen(w)
    df_neu.loc[:, "{}_count".format(w)] = df_neu["text"].map(f_count)
    df_p_neu.loc[:, "{}_count".format(w)] = df_p_neu["text"].map(f_count)
    print(w, np.round(df_neu["{}_count".format(w)].mean()*100,2), np.round(df_p_neu["{}_count".format(w)].mean()*100,2))

also 3.73 0.0
because 6.81 6.81
popular 0.8 0.8
many 4.75 4.75
most 4.56 4.56


### Entailment

In [23]:
ent_art_t, [(syn_noum_dict_inv[w], w) for w in ent_art if w in values]

([('various', 'assorted')], [('respective', 'various')])

In [24]:
for w in ent_art:
    f_count = count_gen(w)
    df_ent.loc[:, "{}_count".format(w)] = df_ent["text"].map(f_count)
    df_p_ent.loc[:, "{}_count".format(w)] = df_p_ent["text"].map(f_count)
    print(w, np.round(df_ent["{}_count".format(w)].mean()*100,2), np.round(df_p_ent["{}_count".format(w)].mean()*100,2))

some 4.55 4.55
yes 0.73 0.73
something 1.8 1.8
sometimes 0.71 0.71
various 0.48 0.0
